## Guider mode quick look
This is intended to unpack the rawStamps from guider mode FITS files and 
give a quick look of a single stamp for one CCD. \
Craig Lage - 16-Apr-25

In [ ]:
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
from matplotlib.backends.backend_pdf import PdfPages

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCol = hdr0['ROICOL']
    roiRow = hdr0['ROIROW']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

# Build a PDF with all of the single stamps

In [ ]:
dayObs = 20250417
seqNum = 599

expId = int(f"{dayObs}{seqNum:05d}")
pdf = PdfPages(f"/home/c/cslage/u/Guider_Mode/Guider_Mode_Stamps_{expId}.pdf")
fig = plt.figure(figsize=(10,10))
rafts = ['R00', 'R04', 'R40', 'R44']
ccds = ['SG0', 'SG1']
n = 0 # Frame 
for raft in rafts:
    for ccd in ccds:
        filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
        
        rp = ResourcePath(filename)
        with rp.open(mode="rb") as f:
            hdu_list = fits.open(f)
        [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
        hdrn = hdu_list[hduNum].header
        timestamp = hdrn['STMPTIME']
        hduNum = 2 * n + 1
        image_out = unpackStamps(hduNum)
        for seg in range(16):
            ax = fig.subplots(1,1)
            roiarr = image_out[seg]
            ax.set_title(f"Guider mode {expId} {raft}_{ccd}_{seg}, \n Frame 1 {timestamp}", fontsize=18) 
            ax.imshow(roiarr, interpolation='nearest', origin='lower', cmap='Greys')
            pdf.savefig(fig)
            plt.clf()
        print(f"Finished {raft}_{ccd}")
pdf.close()
